In [1]:
import pickle
import torch
import numpy as np
from gensim.models import KeyedVectors
from SKIP_THOUGHTS_ARCH import SkipThoughtsNN
import VOCABULARY_EXPANSION
from VOCABULARY_EXPANSION import VocabularyExpansionNN
from VOCABULARY_EXPANSION import GetSTModelWordEmbedding
from VOCABULARY_EXPANSION import GetW2VModelWordEmbedding
import SEMANTIC_RELATEDNESS
from SEMANTIC_RELATEDNESS import SemanticRelatednessNN
from SEMANTIC_RELATEDNESS import GetEmbedding
from SEMANTIC_RELATEDNESS import GetThought
from SEMANTIC_RELATEDNESS import GetSTModelWordEmbeddingSize
from SEMANTIC_RELATEDNESS import GetSTModelThoughtSize

def LoadModelsAndUtils ( loc = 'MODELS' ) :
    global ST_MODEL, VE_MODEL, SR_MODEL, WORD_2_VEC_MODEL, WORD_TO_ID, ID_TO_WORD, ST_VOCAB, W2V_VOCAB
    ST_MODEL = torch.load(loc+'/SKIP_THOUGHTS_NN')
    ST_MODEL.eval()
    VE_MODEL = torch.load(loc+'/VOCABULARY_EXPANSION_NN')
    VE_MODEL.eval()
    SR_MODEL = torch.load(loc+'/SEMANTIC_RELATEDNESS_NN')
    SR_MODEL.eval()

    limit = int(1e7) # 10 Million words
    WORD_2_VEC_MODEL = KeyedVectors.load_word2vec_format(loc+'/GoogleNews-vectors-negative300.bin.gz', 
                                                         binary = True, limit = limit)
    
    file = open(loc+'/UTILS/SKIP_THOUGHTS_NN_WORD2ID', 'rb')
    WORD_TO_ID = pickle.load(file)
    file.close()

    file = open(loc+'/UTILS/SKIP_THOUGHTS_NN_ID2WORD', 'rb')
    ID_TO_WORD = pickle.load(file)
    file.close()

    file = open(loc+'/UTILS/VOCAB_EXPANSION_NN_ST_VOCAB', 'rb')
    ST_VOCAB = pickle.load(file)
    file.close()

    file = open(loc+'/UTILS/VOCAB_EXPANSION_NN_W2V_VOCAB', 'rb')
    W2V_VOCAB = pickle.load(file)
    file.close()

    VOCABULARY_EXPANSION.ST_MODEL = ST_MODEL
    VOCABULARY_EXPANSION.WORD_2_VEC_MODEL = WORD_2_VEC_MODEL
    VOCABULARY_EXPANSION.WORD_TO_ID = WORD_TO_ID
    VOCABULARY_EXPANSION.ID_TO_WORD = ID_TO_WORD
    VOCABULARY_EXPANSION.ST_VOCAB = ST_VOCAB
    VOCABULARY_EXPANSION.W2V_VOCAB = W2V_VOCAB

    SEMANTIC_RELATEDNESS.ST_MODEL = ST_MODEL
    SEMANTIC_RELATEDNESS.ST_VOCAB = ST_VOCAB
    SEMANTIC_RELATEDNESS.W2V_VOCAB = W2V_VOCAB
    SEMANTIC_RELATEDNESS.VE_MODEL = VE_MODEL

def LoadTestDataset ( ) :
    global TEST_SICK_DATA
    file = open('DATA/SICK_TEST_DATA', 'rb')
    TEST_SICK_DATA = pickle.load(file)
    file.close()

def GetSentencePairVector ( sent1 , sent2 ) :
    thought1 = GetThought(sent1)
    if thought1 is None : return None
    thought2 = GetThought(sent2)
    if thought2 is None : return None
    prod = thought1 * thought2
    diff = torch.abs(thought1 - thought2)
    vec = torch.cat([prod, diff], 0)
    return vec

def GetSimilarityRelationScore ( sent1 , sent2 ) :
    vec = GetSentencePairVector(sent1, sent2)
    if vec is None : return None
    inp = torch.stack([vec])
    score_dist = SR_MODEL(inp)[0]
    scores = torch.tensor([1., 2., 3., 4., 5.])
    return round(torch.dot(scores, score_dist).item(), 3)

def PredictScores ( dataset ) :
    true_scores = list()
    predicted_scores = list()
    for sample in TEST_SICK_DATA :
        _, sent1, sent2, true = sample.values()
        pred = GetSimilarityRelationScore(sent1, sent2)
        if pred is None : continue
        true_scores.append(true)
        predicted_scores.append(pred)
    true_scores = np.array(true_scores)
    predicted_scores = np.array(predicted_scores)
    return predicted_scores, true_scores

def GetMSE ( x , y ) :
    return ((x - y) ** 2).mean()

def GetPearsonsCoeff ( x , y ) :
    x_mean = x.mean()
    y_mean = y.mean()
    numer = (x - x_mean) * (y - y_mean)
    numer = numer.sum()
    denom = ((x - x_mean)**2).sum() * ((y - y_mean)**2).sum()
    denom = denom ** 0.5
    return numer / denom

def GetSpearmansCoeff ( x , y ) :
    rank_x = list(enumerate(x))
    rank_x.sort(key = lambda x : -x[1])
    rank_x = [first for first, second in rank_x]
    rank_x = list(enumerate(rank_x))
    rank_x.sort(key = lambda x : x[1])
    rank_x = [first + 1 for first, second in rank_x]
    x = np.array(rank_x)
    
    rank_y = list(enumerate(y))
    rank_y.sort(key = lambda y : -y[1])
    rank_y = [first for first, second in rank_y]
    rank_y = list(enumerate(rank_y))
    rank_y.sort(key = lambda y : y[1])
    rank_y = [first + 1 for first, second in rank_y]
    y = np.array(rank_y)
    
    n = x.shape[0]
    coeff = (x - y) ** 2
    coeff = coeff / (n * (n*n - 1))
    coeff = 6 * coeff.sum()
    return coeff

In [2]:
LoadModelsAndUtils()
LoadTestDataset()
SEMANTIC_RELATEDNESS.WORD_EMBED_SIZE = GetSTModelWordEmbeddingSize()
SEMANTIC_RELATEDNESS.THOUGHT_SIZE = GetSTModelThoughtSize()

## Evaluation On SICK Dataset
The performance of the *Skip-Thoughts Model* with *expanded vocabulary* is checked on *SICK* dataset. Given two sentences, our goal is to produce a score of how semantically related these sentences are and hence compare the score generated by our model with the human generated score given in the *SICK* dataset. The evaluation metrics used are *Mean Sqaured Error*, *Pearson’s coefficient r* and *Spearman’s coefficient ρ*.

In [3]:
predicted_scores, true_scores = PredictScores(TEST_SICK_DATA)

In [4]:
GetMSE(predicted_scores, true_scores)

0.809838439954597

In [5]:
GetPearsonsCoeff(predicted_scores, true_scores)

0.5465874649502056

In [6]:
GetSpearmansCoeff(predicted_scores, true_scores)

0.506588496228052

## Analogical Tasks
The performance of the *Skip-Thoughts Model* with *expanded vocabulary* is checked on how well is it able to quantify the semantic relatedness between two sentences. Given two sentences, a score between 1 and 5 should be generated that is indicative of the intensity of relationship between the sentences.

In [7]:
sentence_1 = "A little girl is looking at a woman in costume"
sentence_2 = "A young girl is looking at a woman in costume"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.877

In [8]:
sentence_1 = "A little girl is looking at a woman in costume"
sentence_2 = "The little girl is looking at a man in costume"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.864

In [9]:
sentence_1 = "A little girl is looking at a woman in costume"
sentence_2 = "A little girl in costume looks like a woman"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.035

In [10]:
sentence_1 = "A sea turtle is hunting for fish"
sentence_2 = "A sea turtle is hunting for food"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.525

In [11]:
sentence_1 = "A sea turtle is not hunting for fish"
sentence_2 = "A sea turtle is hunting for food"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.414

In [12]:
sentence_1 = "A man is driving a car"
sentence_2 = "The car is being driven by a man"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.96

In [13]:
sentence_1 = "There is no man driving the car"
sentence_2 = "A man is driving a car"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.475

In [14]:
sentence_1 = "A large duck is flying over a rocky stream"
sentence_2 = "A duck, which is large, is flying over a rocky stream"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.644

In [15]:
sentence_1 = "A large duck is flying over a rocky stream"
sentence_2 = "A large stream is full of rocks, ducks and flies"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.978

In [16]:
sentence_1 = "A person is performing acrobatics on a motorcycle"
sentence_2 = "A person is performing tricks on a motorcycle"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.082

In [17]:
sentence_1 = "The performer is tricking a person on a motorcycle"
sentence_2 = "A person is performing tricks on a motorcycle"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.094

In [18]:
sentence_1 = "Someone is pouring ingredients into a po"
sentence_2 = "Someone is adding ingredients to a pot"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.437

In [19]:
sentence_1 = "Nobody is pouring ingredients into a pot"
sentence_2 = "Someone is pouring ingredients into a pot"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.638

In [20]:
sentence_1 = "Someone is pouring ingredients into a pot"
sentence_2 = "A man is removing vegetables from a pot"
GetSimilarityRelationScore(sentence_1, sentence_2)

4.2

In [21]:
sentence_1 = "I do not have enough money for this trip"
sentence_2 = "His parents have more than enough money for this trip"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.726

In [22]:
sentence_1 = "I love dogs and she hates cats"
sentence_2 = "She loves dogs and I hate cats"
GetSimilarityRelationScore(sentence_1, sentence_2)

2.988

In [23]:
sentence_1 = "I love dogs and she hates cats"
sentence_2 = "She loves badminton and I hate tennis"
GetSimilarityRelationScore(sentence_1, sentence_2)

2.372

In [24]:
sentence_1 = "A man is playing the guitar loudly"
sentence_2 = "A flute isn't being played by a man"
GetSimilarityRelationScore(sentence_1, sentence_2)

2.695

In [25]:
sentence_1 = "I love driving"
sentence_2 = "I hated driving in the past and now I don't"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.636

In [26]:
sentence_1 = "An onion is being chopped"
sentence_2 = "Someone is holding a hedgehog"
GetSimilarityRelationScore(sentence_1, sentence_2)

1.027

In [27]:
sentence_1 = "The girl is carrying a sign and a group of people is following her"
sentence_2 = "A young girl in a blue shirt is walking on the sidewalk and holding up a pink sign"
GetSimilarityRelationScore(sentence_1, sentence_2)

3.927

In [28]:
sentence_1 = "There is no woman beating two eggs in a bowl and using a whisk made of wire"
sentence_2 = "A man wearing a dyed black shirt is sitting at the table and laughing"
GetSimilarityRelationScore(sentence_1, sentence_2)

1.751

In [29]:
sentence_1 = "A woman is eating a piece of meat"
sentence_2 = "A man is loading a rifle with bullets"
GetSimilarityRelationScore(sentence_1, sentence_2)

1.048